In [9]:
import tensorflow as tf
import tensorflow_gnn as tfgnn
from importlib import reload
from sys import modules
from schema import TYPE_SPEC
from generators import load_dataset_from_data

In [10]:
reload(modules["schema"])
from schema import TYPE_SPEC
spec = TYPE_SPEC["mnist_graph"]

In [11]:
input = tf.keras.layers.Input(type_spec=spec)
gnn = tfgnn.keras.ConvGNNBuilder(
    lambda edge: tfgnn.keras.layers.SimpleConvolution(
        tf.keras.layers.Dense(32)
    ),
    lambda node: tfgnn.keras.layers.NextStateFromConcat(
        tf.keras.layers.Dense(32),
    ),
    )
hidden = gnn.Convolve()(input)
# hidden = tfgnn.keras.layers.Pool(
#     tag=tfgnn.SOURCE,
#     edge_set_name="connected",
#     reduce_type="sum"
# )(hidden)
hidden = tfgnn.keras.layers.Readout(node_set_name="pixel")(hidden)
hidden = tf.keras.layers.Dense(120, activation="tanh")(hidden)
output = tf.keras.layers.Dense(10, activation="softmax")(hidden)

model = tf.keras.Model(input, output, name = "gnn")
model.summary()

Model: "gnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [()]                      0         
                                                                 
 graph_update_1 (GraphUpdate  ()                       1184      
 )                                                               
                                                                 
 readout_1 (Readout)         (784, 32)                 0         
                                                                 
 dense_2 (Dense)             (784, 120)                3960      
                                                                 
 dense_3 (Dense)             (784, 10)                 1210      
                                                                 
Total params: 6,354
Trainable params: 6,354
Non-trainable params: 0
_____________________________________________________________

In [12]:
model.compile(optimizer="adam", loss=tf.keras.losses.categorical_crossentropy, metrics=["accuracy"])

In [13]:
reload(modules["generators"])
from generators import load_dataset_from_data

(trn_img, trn_lbl), _ = tf.keras.datasets.mnist.load_data()
trn_img = trn_img/255
trn_graph = load_dataset_from_data(trn_img, trn_lbl, 32, spec)

In [14]:
print(type(trn_graph))
print(len(list(trn_graph.take(1))))

<class 'tensorflow.python.data.ops.dataset_ops.CacheDataset'>


2022-03-01 23:47:30.268638: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1


In [15]:
model.fit(trn_graph, epochs=10)

Epoch 1/10


ValueError: in user code:

    File "/home/thomas/Documents/tfgnn-mnist/env/lib/python3.9/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/thomas/Documents/tfgnn-mnist/env/lib/python3.9/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/thomas/Documents/tfgnn-mnist/env/lib/python3.9/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/thomas/Documents/tfgnn-mnist/env/lib/python3.9/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/thomas/Documents/tfgnn-mnist/env/lib/python3.9/site-packages/keras/engine/training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "/home/thomas/Documents/tfgnn-mnist/env/lib/pyth